So, I'm doing this project of extracting date data from a text file with very messy format. The date formate is very messey, and I suppose to use regex and use some data cleaning to deal with it. 
Here is a list of some of the variants in this dataset:
04/20/2009; 04/20/09; 4/20/09; 4/3/09
Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
Feb 2009; Sep 2009; Oct 2010
6/2008; 12/2009
2009; 2010

In [12]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df

0           03/25/93 Total time of visit (in minutes):\n
1                         6/18/85 Primary Care Doctor:\n
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                  7 on 9/27/75 Audit C Score Current:\n
4      2/6/96 sleep studyPain Treatment Pain Level (N...
5                      .Per 7/06/79 Movement D/O note:\n
6      4, 5/18/78 Patient's thoughts about current su...
7      10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                           3/7/86 SOS-10 Total Score:\n
9               (4/10/71)Score-1Audit C Score Current:\n
10     (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                         4/09/75 SOS-10 Total Score:\n
12     8/01/98 Communication with referring physician...
13     1/26/72 Communication with referring physician...
14     5/24/1990 CPT Code: 90792: With medical servic...
15     1/25/2011 CPT Code: 90792: With medical servic...
16           4/12/82 Total time of visit (in minutes):\n
17          1; 10/13/1976 Audit

After loading the data I did some replacement to clean up a little

In [14]:
df = df.str.replace(r'[,.]', '')
df = df.str.replace(r'[-]', '/')
df = df.str.replace(r'(\d \d{2} \d{2})', lambda x: x.groups()[0][0]+'/'+x.groups()[0][2:-3]+'/19'+x.groups()[0][-2:])
df = df.str.replace(r'(\d{,2}/\d{,2}/\d{2})', lambda x: x.groups()[0][:-2]+'19'+x.groups()[0][-2:])

Then I extract out the relevent information based on the format

In [15]:
t1 = df.str.extractall(r'(?P<month>\d{1,2})/*(?P<day>\d{,2})/(?P<year>[12]\d{3})').fillna(1)
t1.reset_index(inplace =True)
t1_index = t1['level_0']
t2 = df.str.extractall(r'(?P<day>\d{,2})? *(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*) (?P<year>\d{4})').fillna(1)
t2.reset_index(inplace =True)
t2_index = t2['level_0']
t3 = df.str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)?[ ](?P<day>\d{,2})?[ ](?P<year>\d{4})').fillna(1)
t3.reset_index(inplace =True)
t3_index = t3['level_0']

In [16]:
df = df.drop(t1_index)
df = df.drop(t2_index)
df = df.drop(t3_index)

In [21]:
t4 = df.str.extractall(r'(?P<year>[12]\d{3})(?P<month>)(?P<day>)').fillna(1)
t4.reset_index(inplace = True)
t4_index = t4['level_0']

In [22]:
t4

,level_0,match,year,month,day
0,455,0,1984,1.0,1.0
1,456,0,2000,1.0,1.0
2,457,0,2001,1.0,1.0
3,458,0,1982,1.0,1.0
4,459,0,1998,1.0,1.0
5,460,0,2012,1.0,1.0
6,462,0,1988,1.0,1.0
7,463,0,2014,1.0,1.0
8,464,0,2016,1.0,1.0
9,466,0,1981,1.0,1.0


To check overlaps

In [23]:
overlap_t14 = [x for x in t4_index if x in t1_index]
overlap_t24 = [x for x in t4_index if x in t2_index]
overlap_t34 = [x for x in t4_index if x in t3_index]
overlap_all = []
overlap_all.extend(overlap_t14)
overlap_all.extend(overlap_t24)
overlap_all.extend(overlap_t34)

In [24]:
overlap_all

[]

Put dataframes together and replace the month

In [26]:
Data = pd.concat([t1,t2,t3,t4]).fillna(1)

In [27]:
month_dict = dict({'Jan': 1, 'Janaury': 1,'January': 1,'Feb': 2, 'February': 2,  'Mar': 3, 'March': 3,'Apr': 4, 'April':4, 
                   'May': 5, 'Jun': 6, 'June': 6,   'Jul': 7,'July': 7, 'Aug': 8, 'August': 8, 'Decemeber':12, 
                   'Sep': 9, 'September': 9,'Oct': 10, 'October': 10,'Nov': 11,'November': 11,'Dec': 12, 'December': 12})

In [28]:
for x in month_dict:
    Data.month.replace(x, month_dict[x], inplace = True)

496

Since there are two entry that's incorrectly captured (I found these two the month and day doesn't make sense and went back to check on the data), I will remove the two entry

In [31]:
print(Data.iloc[11]) 

day           1
level_0      10
match         1
month        16
year       1922
Name: 11, dtype: object


In [30]:
Data.iloc[74]

day          36
level_0      72
match         1
month        49
year       1930
Name: 74, dtype: object

In [33]:
to_drop = [11,74]
Data = Data.drop(Data.index[to_drop])

In [35]:
len(Data)

496

I checked the lenth of the Data at this point, seems like I am missing 4 entries, since the total lenth should be 500. however I don't see where is the miss. when I check it I got this:

In [36]:
r500 = range(500)
miss = [x for x in r500 if x not in Data['level_0']]
miss

[11,
 74,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,


My first thought was, hey I only miss 4 entries, why is this suggestiong I am missing a lot. so then I sort the Data and try to get a clear view

In [37]:
r500 = range(500)
l500 = sorted(Data['level_0'])
miss = [x for x in r500 if x not in l500]
miss

[136, 205, 233, 468]

and it give me the answer that I want, my next question is what happend to 136, I clear remember seeing this entry in t2. 

In [38]:
136 in Data['level_0']

True

In [39]:
136 in sorted(['level_0'])

False

So at this point, seems like sorted() did something that I wasn't expecting. and I really don't know why. I went and did some search, didn't find something very useful. So I try to track the item 136 and see when and where it disappeared. I first make another copy of the Data before I remove the two error entry:

In [41]:
Data1 = pd.concat([t1,t2,t3,t4]).fillna(1)
len(Data1)

502

Something interesting happen right here, before I remove the two entry the length of the Data frame is 502, and after I remove it become 496, so when I thought I only removed two entries, I really removed 6! 

In [44]:
Data1['level_0'][11]

11     10
11    136
11    205
11    468
Name: level_0, dtype: int64

In [45]:
Data1.iloc[11]

day           1
level_0      10
match         1
month        16
year       1922
Name: 11, dtype: object

In [46]:
Data1.iloc[136]

day           1
level_0     352
match         0
month         3
year       1986
Name: 136, dtype: object

So here comes the problem when I use iloc, it only refer to one entry, when I use drop to drop the entry, it dropped not only to that entry but also other data that associate to that index, when I call Data1['level_0'][11] 4 entry was return all associated with same index. 

Another problem I would like a more clear understanding would be:
136 in Data['level_0'] returns True
136 in sorted(Data['level_0']) returns False.
I don't gey why the first commend would return True, since I have accendentally removed the element. 

My understanding is Data['level_0'] only check the first one within the memory slot, where group by actrually have to iterate through all the entry so it can see the whole entry. 
If someone know why this is happening please let me know